## Es. 1:
Scrivere una funzione `sum n` che calcoli **ricorsivamente** la somma di tutti i numeri da 0 a n. Usarla poi per calcolare `sum 1000000`

In [ ]:
let rec sum n = 
 match n with 
 | 0 -> 0
 | n -> n + sum (n - 1);;
 
 sum 4;;
 sum 1000000;;

In [3]:
let sum2 n =
    let rec aux n tot =
        match n with
        | 0 -> tot
        | n -> aux (n-1) (tot + n)
    in aux n 0;;
    
sum2 1000000;;

val sum2 : int -> int = <fun>


- : int = 500000500000


## Es. 2:
Scrivere una funzione `square_evens l` che prende una lista  `l` e restituisce una lista `l'`, contenente **solo** gli elementi pari di `l`, elevati al quadrato.

    square_evens [4; 1; 5; 4; 10] = [16; 16; 100]
 
 Scrivere due versioni di `square_evens`, una usando ricorsione e pattern matching, una usando funzioni di ordine superiore (non necessariamente `fold`).

In [14]:
(*Soluzione 1, usando ricorsione e pattern matching*)
let rec square_evens l = 
    match l with 
    | [] -> []
    | x :: xs -> if (x mod 2 = 0) 
                    then x*x :: square_evens xs
                    else square_evens xs;;
                    
    
(*Soluzione 2, usando filter e map*)
let square_evens2 l = 
    let filtered = List.filter (fun x -> (x mod 2) = 0) l in
    let squared = List.map (fun x -> x*x) filtered in
    squared;;

(*Soluzione 3, usando fold right*)
let square_evens3 l = 
    let f el acc = if (el mod 2) = 0 
                    then el*el :: acc
                    else acc
    in
    List.fold_right f l [];;

square_evens3 [4; 1; 5; 4; 10];;

val square_evens : int list -> int list = <fun>


val square_evens2 : int list -> int list = <fun>


val square_evens3 : int list -> int list = <fun>


- : int list = [16; 16; 100]


## Es. 3: 

Scrivere una funzione `drop n l` che prende un intero e una lista, e scarta tutti gli elementi di `l` con **indice** multiplo di n. Usare sia la ricorsione che `fold_left` o `fold_right`.

    drop 2 [1; 2; 3; 17; 42] = [1; 3; 42]

## Es. 4:
Scrivere una funzione `ordered l` che restituisca `true` se e solo se `l` è una lista ordinata.

    ordered [1; 2; 4; 10] = true
    ordered [1; 3; 2; 10] = false